# Custom Data Configurations

In [ ]:
%pip install rdflib --quiet
import yfiles_jupyter_graphs_for_sparql 
from yfiles_jupyter_graphs_for_sparql import SparqlGraphWidget

You can also open this notebook in Google Colab when Google Colab's custom widget manager is enabled:

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [ ]:
sgw = SparqlGraphWidget(data="https://ld.cultural.jp/sparql")

## Configure data mappings
Configurable values for the object visualization: text, color, size, styles, property, coordinate and type.

Configurable values for the subject visualization: text, color, size, styles, property, coordinate and type.

Configurable values for the predicate visualization: text, color, thickness_factor and property.

When adding a configuration, you always need to specify the type of predicate of the object, subject or predicate you want to adjust. (Here 'member' or 'writer'), additionally you can define the new values for the above mentioned properties. The values may either be:

A lambda the resolves the node or relationship to a specific value.
A constant value which is directly used.
A property-key that points to the value that should be used for the configuration.

In [ ]:
sgw = SparqlGraphWidget(data='beatles.ttl', limit=20)
query = """
SELECT ?sub ?p ?ob
WHERE {
    ?sub ?p ?ob .
}
"""
sgw.add_object_configuration('member', color='purple')
sgw.add_subject_configuration('member', color='blue')
sgw.add_predicate_configuration('writer', color='green')
sgw.show_query(query)

### Delete a configuration 

Configurations are stored within the class instance and must be explicitly removed to no longer be applied.

In [ ]:
sgw.del_predicate_configuration('writer')
sgw.show_query(query)

In [ ]:
sgw.add_parent_configuration('member', True)
sgw.add_subject_configuration('track', parent_configuration='type')
sgw.show_query("""
SELECT ?sub ?p ?ob
WHERE {
    ?sub ?p ?ob .
}
""")

In [ ]:
sgw._object_configurations

In [ ]:
sgw._subject_configurations

In [ ]:
sgw._parent_configurations